In [1]:
import importlib.util
import sys
loc = '/afs/ciemat.es/user/m/martialc/public/muTel_v4/muTel/src/muTel/__init__.py'
# loc = r'F:\Clase\03 - Máster\TFM\Programa\muTel\src\muTel\__init__.py'
src = 'muTel'
spec = importlib.util.spec_from_file_location(src, loc)
foo = importlib.util.module_from_spec(spec)
sys.modules[src] = foo
spec.loader.exec_module(foo)
%load_ext autoreload


In [2]:
import dask.dataframe as dd
import pyarrow.parquet as pq
import numpy as np

In [4]:
file = "/afs/ciemat.es/user/m/martialc/public/muTel_v4/muTel/data/run_588.parquet"

In [5]:
pq.read_table(file).schema.metadata

{b'run': b'588',
 b'date': b'NA',
 b'pandas': b'{"index_columns": ["__index_level_0__"], "column_indexes": [{"name": null, "field_name": null, "pandas_type": "unicode", "numpy_type": "object", "metadata": {"encoding": "UTF-8"}}], "columns": [{"name": "EventNr", "field_name": "EventNr", "pandas_type": "int32", "numpy_type": "int32", "metadata": null}, {"name": "GEO", "field_name": "GEO", "pandas_type": "int8", "numpy_type": "int8", "metadata": null}, {"name": "hit", "field_name": "hit", "pandas_type": "int16", "numpy_type": "int16", "metadata": null}, {"name": "channel", "field_name": "channel", "pandas_type": "int8", "numpy_type": "int8", "metadata": null}, {"name": "chamber", "field_name": "chamber", "pandas_type": "int64", "numpy_type": "int64", "metadata": null}, {"name": "sl", "field_name": "sl", "pandas_type": "int8", "numpy_type": "int8", "metadata": null}, {"name": "layer", "field_name": "layer", "pandas_type": "int8", "numpy_type": "int8", "metadata": null}, {"name": "cell", "f

In [6]:
ddf = dd.read_parquet(
            file,
            engine='pyarrow',
            calculate_divisions=True
        ) 
ddf.compute_current_divisions()

(2, 795023)

In [7]:
npartitions=5
max_enr = ddf.EventNr.max().compute()
divs_enr = np.floor(np.arange(npartitions+1)/npartitions*max_enr)
    
divs_idx = list(map(lambda x: ddf.where(ddf.EventNr == x).dropna().index.min().compute(), divs_enr))  

divs_idx[0] = ddf.compute_current_divisions()[0]
divs_idx[-1] = ddf.compute_current_divisions()[-1]
# ddf = ddf.repartition(divisions=tuple(divs_idx))

In [8]:
ddf_rep_dask = ddf.repartition(npartitions=5)

In [9]:
ddf_rep_mine = ddf.repartition(divisions=tuple(divs_idx))

In [10]:
display(ddf_rep_mine.compute_current_divisions())
display(ddf_rep_dask.compute_current_divisions())

(2, 170560, 328845, 490229, 649636, 795023)

(2, 158467, 317706, 476791, 636041, 795023)

In [11]:
ddf_rep_dask.repartition(divisions=tuple(divs_idx)).head()

ValueError: left side of old and new divisions are different

In [62]:
sample = np.random.randint(0,ddf.EventNr.max(),100)

ValueError: left side of old and new divisions are different